# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [32]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [33]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [34]:
from iex_secrets import IEX_CLOUD_API_TOKEN
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2492418048212,
 'week52high': 183.28,
 'week52low': 132.3,
 'week52highSplitAdjustOnly': 191.76,
 'week52lowSplitAdjustOnly': 135.11,
 'week52change': 0.0390910128369899,
 'sharesOutstanding': 16748966125,
 'float': 0,
 'avg10Volume': 78455243,
 'avg30Volume': 77220464,
 'day200MovingAvg': 164.89,
 'day50MovingAvg': 146.76,
 'employees': 153174,
 'ttmEPS': 6.33,
 'ttmDividendRate': 0.9173566583121574,
 'dividendYield': 0.00589878143161567,
 'nextDividendDate': '',
 'exDividendDate': '2022-04-23',
 'nextEarningsDate': '2022-07-25',
 'peRatio': 24.47533391584377,
 'beta': 1.2836000681452,
 'maxChangePercent': 59.55661702765747,
 'year5ChangePercent': 3.295891460295218,
 'year2ChangePercent': 0.593732602582261,
 'year1ChangePercent': 0.06679465466318926,
 'ytdChangePercent': -0.1526086301187472,
 'month6ChangePercent': -0.09136589213946224,
 'month3ChangePercent': -0.09671897109846331,
 'month1ChangePercent': 0.1535891803060763,
 'day30ChangePerc

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [35]:
data['year1ChangePercent']

0.06679465466318926

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [36]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [37]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url  = f'https://sandbox.iexapis.com/stable/stock/market/batch?token={IEX_CLOUD_API_TOKEN}&types=price,stats&symbols={symbol_string}'
    print(batch_api_url)
    data = requests.get(batch_api_url).json()
    
    # for symbol in symbol_string.split(','):
    for symbol in data:
        # print(data[symbol]['price'])
        # print(data[symbol]['stats'])
        final_dataframe = final_dataframe.append(
            pd.Series(
                [symbol, data[symbol]['price'], data[symbol]['stats']['year1ChangePercent'], 'N/A'],
                index = my_columns,
            ),
            ignore_index = True
        )
final_dataframe

https://sandbox.iexapis.com/stable/stock/market/batch?token=Tpk_0020df4fd968417db3419f6708aba4fa&types=price,stats&symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
https://sandbox.iexapis.com/stable/stock/market/batch?token=Tpk_0020df4fd968417db3419f6708aba4fa&types=price,stats&symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,F

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,122.24,-0.190535,N/A
1,AAL,15.80,-0.20413,N/A
2,AAP,194.16,-0.050783,N/A
3,AAPL,158.96,0.068615,N/A
4,ABBV,150.50,0.353969,N/A
...,...,...,...,...
496,YUM,125.09,0.076309,N/A
497,ZBH,112.00,-0.293732,N/A
498,ZBRA,336.27,-0.403832,N/A
499,ZION,55.81,0.105097,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [38]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True, drop=True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.110,2.394596,N/A
1,OXY,63.860,1.582951,N/A
2,DVN,58.780,1.514949,N/A
3,MRO,23.240,1.064952,N/A
4,APA,35.421,1.051702,N/A
5,CF,85.670,0.948372,N/A
6,VLO,111.560,0.875952,N/A
7,MPC,93.120,0.806155,N/A
8,MCK,336.700,0.765689,N/A
9,DLTR,175.740,0.748245,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [39]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()
print(portfolio_size)

10000000


In [40]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe
    

/Users/luxiaotong/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.110,2.394596,2435
1,OXY,63.860,1.582951,3131
2,DVN,58.780,1.514949,3402
3,MRO,23.240,1.064952,8605
4,APA,35.421,1.051702,5646
5,CF,85.670,0.948372,2334
6,VLO,111.560,0.875952,1792
7,MPC,93.120,0.806155,2147
8,MCK,336.700,0.765689,594
9,DLTR,175.740,0.748245,1138


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [41]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqm_dataframe = pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings:
    batch_api_url  = f'https://sandbox.iexapis.com/stable/stock/market/batch?token={IEX_CLOUD_API_TOKEN}&types=price,stats&symbols={symbol_string}'
    print(batch_api_url)
    data = requests.get(batch_api_url).json()
    for symbol in data:
        # print(data[symbol]['price'])
        # print(data[symbol]['stats'])
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A',
                ],
                index = hqm_columns,
            ),
            ignore_index = True
        )
hqm_dataframe

https://sandbox.iexapis.com/stable/stock/market/batch?token=Tpk_0020df4fd968417db3419f6708aba4fa&types=price,stats&symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
https://sandbox.iexapis.com/stable/stock/market/batch?token=Tpk_0020df4fd968417db3419f6708aba4fa&types=price,stats&symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,F

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,122.77,N/A,-0.192094,N/A,-0.156304,N/A,-0.058808,N/A,0.058457,N/A,N/A
1,AAL,15.43,N/A,-0.204581,N/A,-0.130208,N/A,-0.230201,N/A,0.178404,N/A,N/A
2,AAP,197.21,N/A,-0.050421,N/A,-0.180419,N/A,-0.141703,N/A,0.142902,N/A,N/A
3,AAPL,153.45,N/A,0.067398,N/A,-0.090132,N/A,-0.099321,N/A,0.150351,N/A,N/A
4,ABBV,147.75,N/A,0.355079,N/A,0.129448,N/A,-0.03398,N/A,0.094428,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,122.19,N/A,0.077947,N/A,-0.028706,N/A,-0.034705,N/A,0.099389,N/A,N/A
497,ZBH,111.00,N/A,-0.285394,N/A,-0.129302,N/A,-0.180269,N/A,0.048752,N/A,N/A
498,ZBRA,341.65,N/A,-0.393677,N/A,-0.411593,N/A,-0.240124,N/A,0.084832,N/A,N/A
499,ZION,54.73,N/A,0.102863,N/A,-0.212805,N/A,-0.203453,N/A,0.006808,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [68]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(
                hqm_dataframe[change_col],
                hqm_dataframe.loc[row, change_col]
            )/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,HRB,39.50,506,0.709416,0.92,0.767879,0.96,0.403934,1.0,0.157541,0.82,98.053892
1,COG,23.31,858,0.350178,0.76,0.189537,0.78,0.262377,0.98,0.253659,0.98,97.355289
2,OXY,65.51,305,1.610789,0.98,0.778422,0.98,0.044093,0.58,0.13055,0.64,95.409182
3,CNC,93.63,213,0.282858,0.68,0.178187,0.74,0.065327,0.78,0.197552,0.94,95.409182
4,LLY,322.25,62,0.415343,0.82,0.349597,0.94,0.106751,0.92,0.123389,0.56,94.510978
5,TAP,59.16,338,0.206481,0.5,0.189982,0.8,0.057487,0.68,0.168342,0.88,93.912176
6,UNH,530.70,37,0.323726,0.72,0.160956,0.7,-0.004413,0.32,0.187542,0.92,92.265469
7,CI,276.86,72,0.23069,0.56,0.165712,0.72,0.058912,0.74,0.127969,0.6,92.015968
8,DLTR,174.11,114,0.756664,0.94,0.298778,0.88,-0.009842,0.3,0.135862,0.7,91.916168
9,HUM,486.25,41,0.08508,0.18,0.328585,0.92,0.084971,0.84,0.161665,0.84,91.616766


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [69]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    # print(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,HRB,39.50,506,0.709416,0.92,0.767879,0.96,0.403934,1.0,0.157541,0.82,0.925
1,COG,23.31,858,0.350178,0.76,0.189537,0.78,0.262377,0.98,0.253659,0.98,0.875
2,OXY,65.51,305,1.610789,0.98,0.778422,0.98,0.044093,0.58,0.13055,0.64,0.795
3,CNC,93.63,213,0.282858,0.68,0.178187,0.74,0.065327,0.78,0.197552,0.94,0.785
4,LLY,322.25,62,0.415343,0.82,0.349597,0.94,0.106751,0.92,0.123389,0.56,0.81
5,TAP,59.16,338,0.206481,0.5,0.189982,0.8,0.057487,0.68,0.168342,0.88,0.715
6,UNH,530.70,37,0.323726,0.72,0.160956,0.7,-0.004413,0.32,0.187542,0.92,0.665
7,CI,276.86,72,0.23069,0.56,0.165712,0.72,0.058912,0.74,0.127969,0.6,0.655
8,DLTR,174.11,114,0.756664,0.94,0.298778,0.88,-0.009842,0.3,0.135862,0.7,0.705
9,HUM,486.25,41,0.08508,0.18,0.328585,0.92,0.084971,0.84,0.161665,0.84,0.695


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [70]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,HRB,39.50,506,0.709416,0.92,0.767879,0.96,0.403934,1.0,0.157541,0.82,0.925
1,COG,23.31,858,0.350178,0.76,0.189537,0.78,0.262377,0.98,0.253659,0.98,0.875
2,LLY,322.25,62,0.415343,0.82,0.349597,0.94,0.106751,0.92,0.123389,0.56,0.81
3,OXY,65.51,305,1.610789,0.98,0.778422,0.98,0.044093,0.58,0.13055,0.64,0.795
4,CNC,93.63,213,0.282858,0.68,0.178187,0.74,0.065327,0.78,0.197552,0.94,0.785
5,LB,82.56,242,2.304758,1.0,0.822037,1.0,0.218447,0.96,0.081088,0.1,0.765
6,TAP,59.16,338,0.206481,0.5,0.189982,0.8,0.057487,0.68,0.168342,0.88,0.715
7,DLTR,174.11,114,0.756664,0.94,0.298778,0.88,-0.009842,0.3,0.135862,0.7,0.705
8,HUM,486.25,41,0.08508,0.18,0.328585,0.92,0.084971,0.84,0.161665,0.84,0.695
9,PWR,134.90,148,0.550474,0.88,0.28699,0.86,-0.055754,0.16,0.152326,0.78,0.67


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [50]:
portfolio_input()

In [71]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
# position_size
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,HRB,39.50,506,0.709416,0.92,0.767879,0.96,0.403934,1.0,0.157541,0.82,0.925
1,COG,23.31,858,0.350178,0.76,0.189537,0.78,0.262377,0.98,0.253659,0.98,0.875
2,LLY,322.25,62,0.415343,0.82,0.349597,0.94,0.106751,0.92,0.123389,0.56,0.81
3,OXY,65.51,305,1.610789,0.98,0.778422,0.98,0.044093,0.58,0.13055,0.64,0.795
4,CNC,93.63,213,0.282858,0.68,0.178187,0.74,0.065327,0.78,0.197552,0.94,0.785
5,LB,82.56,242,2.304758,1.0,0.822037,1.0,0.218447,0.96,0.081088,0.1,0.765
6,TAP,59.16,338,0.206481,0.5,0.189982,0.8,0.057487,0.68,0.168342,0.88,0.715
7,DLTR,174.11,114,0.756664,0.94,0.298778,0.88,-0.009842,0.3,0.135862,0.7,0.705
8,HUM,486.25,41,0.08508,0.18,0.328585,0.92,0.084971,0.84,0.161665,0.84,0.695
9,PWR,134.90,148,0.550474,0.88,0.28699,0.86,-0.055754,0.16,0.152326,0.78,0.67


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [72]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [73]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [74]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
writer.save()


## Saving Our Excel Output

As before, saving our Excel output is very easy: